In [1]:
import pandas as pd
import numpy as np

import glob
import os
import sys
import math
from decimal import Decimal, ROUND_HALF_UP

In [2]:
def calc_xy(phi_deg, lambda_deg):
    #(phi_deg, lambda_deg): 変換したい緯度・経度[度]（分・秒でなく小数であることに注意）
    #(phi0_deg, lambda0_deg): 平面直角座標系原点の緯度・経度[度]（分・秒でなく小数であることに注意）
    # 緯度経度・平面直角座標系原点をラジアンに直す
    phi0_deg = 36.0000
    lambda0_deg = 136.0000
    phi_rad = np.deg2rad(phi_deg)
    lambda_rad = np.deg2rad(lambda_deg)
    phi0_rad = np.deg2rad(phi0_deg)
    lambda0_rad = np.deg2rad(lambda0_deg)

    # 補助関数
    def A_array(n):
        A0 = 1 + (n**2)/4. + (n**4)/64.
        A1 = -     (3./2)*( n - (n**3)/8. - (n**5)/64. ) 
        A2 =     (15./16)*( n**2 - (n**4)/4. )
        A3 = -   (35./48)*( n**3 - (5./16)*(n**5) )
        A4 =   (315./512)*( n**4 )
        A5 = -(693./1280)*( n**5 )
        return np.array([A0, A1, A2, A3, A4, A5])
  
    def alpha_array(n):
        a0 = np.nan # dummy
        a1 = (1./2)*n - (2./3)*(n**2) + (5./16)*(n**3) + (41./180)*(n**4) - (127./288)*(n**5)
        a2 = (13./48)*(n**2) - (3./5)*(n**3) + (557./1440)*(n**4) + (281./630)*(n**5)
        a3 = (61./240)*(n**3) - (103./140)*(n**4) + (15061./26880)*(n**5)
        a4 = (49561./161280)*(n**4) - (179./168)*(n**5)
        a5 = (34729./80640)*(n**5)
        return np.array([a0, a1, a2, a3, a4, a5])

    # 定数 (a, F: 世界測地系-測地基準系1980（GRS80）楕円体)
    m0 = 0.9999 
    a = 6378137.
    F = 298.257222101
    
   #各計算
    n = 1. / (2*F - 1)
    A_array = A_array(n)
    alpha_array = alpha_array(n)
        
    A_ = ( (m0*a)/(1.+n) )*A_array[0] # [m]
    S_ = ( (m0*a)/(1.+n) )*( A_array[0]*phi0_rad + np.dot(A_array[1:], np.sin(2*phi0_rad*np.arange(1,6))) ) # [m]
    
    lambda_c = np.cos(lambda_rad - lambda0_rad)
    lambda_s = np.sin(lambda_rad - lambda0_rad)
    
    t = np.sinh( np.arctanh(np.sin(phi_rad)) - ((2*np.sqrt(n)) / (1+n))*np.arctanh(((2*np.sqrt(n)) / (1+n)) * np.sin(phi_rad)) )
    t_ = np.sqrt(1 + t*t)

    xi2  = np.arctan(t / lambda_c) # [rad]
    eta2 = np.arctanh(lambda_s / t_)
    
    # (6) x, yの計算 (x: 変換後の平面直角座標[m] y: 変換後の平面直角座標[m])
    x = A_ * (xi2 + np.sum(np.multiply(alpha_array[1:],
                                       np.multiply(np.sin(2*xi2*np.arange(1,6)),
                                                   np.cosh(2*eta2*np.arange(1,6)))))) - S_ # [m]
    y = A_ * (eta2 + np.sum(np.multiply(alpha_array[1:],
                                        np.multiply(np.cos(2*xi2*np.arange(1,6)),
                                                    np.sinh(2*eta2*np.arange(1,6)))))) # [m]
    # return
    return x, y # [m]

In [3]:
data = pd.read_csv("data/target_point.csv", index_col=0)

In [4]:
data.head()

,北緯,東経,北緯東経,X座標(6系),Y座標(6系),X座標(A基準),Y座標(A基準)
地点,,,,,,,
A,34.735638,135.840736,"34°44'08.3""N 135°50'26.7""E",-140251.8033,-14583.70539,0.000000,0.000000
B,34.735484,135.840736,"34°44'07.7""N 135°50'26.7""E",-140269.5513,-14583.73350,-17.747968,-0.028109
C,34.735477,135.840001,"34°44'07.7""N 135°50'24.0""E",-140269.4437,-14651.49674,-17.640395,-67.791343
D,34.735408,135.840002,"34°44'07.5""N 135°50'24.0""E",-140277.2084,-14651.50909,-25.405131,-67.803698
E,34.735412,135.839942,"34°44'07.5""N 135°50'23.8""E",-140277.1997,-14657.00341,-25.396387,-73.298019


In [5]:
for i in range(len(data)):
    phi_deg = data["北緯"][i].round(5)
    lambda_deg = data["東経"][i].round(5)
#     print(phi_deg, lambda_deg)
    x,y = calc_xy(phi_deg, lambda_deg)
    print(y)
  
    
    

-14583.705393358234
-14583.733502553345
-14651.496736843572
-14651.509091722372
-14657.00341268536


# ⇩"度分"のデータから　"度分秒"を作る

In [6]:
longnitude = []
#経度の計算
for i in range(len(data)):
    long = data["東経"][i]
    rad = int(long)
    sec = ((long - int(long))*60)
    mi = (sec-sec.round())*60
    sec = int(sec.round())
    lo = [str(rad),str(sec),str(mi)] 
    longnitude.append(lo)
print(longnitude[1])
#     print(str(rad) +str(sec) +"'"+ str(mi))

['135', '50', '26.64960000007568']


In [7]:
latitude = []
# 緯度の計算
for i in range(len(data)):
    lat = data["北緯"][i]
    rad = int(lat)
    sec = ((lat - int(lat))*60)
    mi = (sec-sec.round())*60
    sec = int(sec.round())
    la = [str(rad),str(sec),str(mi)] 
    latitude.append(la)

print(latitude[1])


['34', '44', '7.742399999998497']


In [8]:
name = ["A","B","C","D","E"]


In [9]:
location = []
for j in range (4):
    #経度の計算
    longnitude = []
    for i in range(len(data)):
        long = data["東経"][i]
        rad = int(long)
        sec = ((long - int(long))*60)
        mi = (sec-sec.round())*60
        sec = int(sec.round())
        lo = [str(rad),str(sec),str(mi)] 
        longnitude.append(lo)
    
   
    # 緯度の計算
    latitude = []
    for i in range(len(data)):
        lat = data["北緯"][i]
        rad = int(lat)
        sec = ((lat - int(lat))*60)
        mi = (sec-sec.round())*60
        sec = int(sec.round())
        la = [str(rad),str(sec),str(mi)] 
        latitude.append(la)
    
    locate = [latitude[j],longnitude[j]]
    location.append(locate)
    #     print(str(rad) +str(sec) +"'"+ str(mi))
    

In [10]:
print(location[1])

[['34', '44', '7.742399999998497'], ['135', '50', '26.64960000007568']]
